In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
from shapely.geometry import Point
from pyproj import CRS
from scipy.linalg import qr

In [2]:
df_cleaned = pd.read_pickle("outputs/df_cleaned.pkl")
gauges_gdf = gpd.read_file("outputs/gauges.gpkg")
flowlines_gdf = gpd.read_file("outputs/flowlines.gpkg")
usgs_index_df = pd.read_pickle("outputs/usgs_index.pkl")
tx_shp = gpd.read_file("outputs/tx_gulf_shapefile/tx_gulf_bnd.shp").to_crs(4269)
huc6 = gpd.read_file("outputs/huc6_shapefile/huc6_tx_gulf.shp").to_crs(4269)

In [3]:
X = df_cleaned.values.astype(float)
comid = df_cleaned.columns
n_time, n_sites = X.shape
print('X shape:', X.shape)

train_frac = 0.7
split_idx = int(train_frac * n_time)
X_train = X[:split_idx,:]
X_test = X[split_idx:,:]
print('Train shape:', X_train.shape)
print('Test shape :', X_test.shape)

X shape: (4017, 64907)
Train shape: (2811, 64907)
Test shape : (1206, 64907)


In [4]:
# Make a copy to avoid modifying the original
flowlines_copy = flowlines_gdf.copy()
flowlines_copy["COMID"] = flowlines_copy["COMID"].astype(str)

col_ids = pd.Index(df_cleaned.columns.astype(str))
# dictionary for comid and idx
comid_to_col = {cid: j for j, cid in enumerate(col_ids)}

# Filter flowlines to only COMIDs in df_cleaned
flowlines_filtered = flowlines_copy.loc[flowlines_copy["COMID"].isin(comid)].copy()
# map idx to comid
flowlines_filtered["idx"] = flowlines_filtered["COMID"].astype(str).map(comid_to_col).astype(int)

# Reproject to a projected CRS (x,y)
gdf_proj = flowlines_filtered.to_crs(epsg=5070)

# Compute centroids
centroids = gdf_proj.geometry.centroid

# Convert centroids back to geographic CRS (lat/lon)
centroids_ll = gpd.GeoSeries(centroids, crs=gdf_proj.crs).to_crs(epsg=4269)

# Extract lat / lon 
flow_lat = centroids_ll.y.values
flow_lon = centroids_ll.x.values

# Create "(lat, lon)" formatted strings
latlon_str = [f"({lat}, {lon})" for lat, lon in zip(flow_lat, flow_lon)]

# Build final DataFrame (same order as flowlines_gdf)
flow_lat_lon = flowlines_filtered.assign(
    lat=flow_lat,
    lon=flow_lon,
    lat_lon=latlon_str
)[["COMID", "lat_lon","idx"]]
# need to save crs for lat lon coord_crs = 
flow_lat_lon

,COMID,lat_lon,idx
0,5256723,"(32.61868276306611, -95.50137088386384)",26936
1,5256353,"(32.63013428977702, -95.51512370866801)",26782
2,5256289,"(32.64981553314542, -95.53823507416166)",26750
3,5256227,"(32.661436989062686, -95.5524962744192)",26720
4,5256215,"(32.668785323658845, -95.54849766843817)",26714
...,...,...,...
68834,210053,"(26.02768204709921, -97.604964548574)",371
68835,210055,"(26.02905084838759, -97.60635442599998)",372
68836,210057,"(26.02920018003503, -97.60606247351534)",373
68837,942110021,"(26.166418341363304, -97.46932823893809)",64897


In [5]:
usgs_index_df_valid = usgs_index_df.loc[usgs_index_df['COMID'].isin(comid.astype(int))]
usgs_index_filter = usgs_index_df_valid[np.abs(usgs_index_df_valid['COMID'])!=9999]
usgs_index_lon = usgs_index_filter['Long_snap'].values
usgs_index_lat = usgs_index_filter['Lat_snap'].values
usgs_index_coord = [f"({lat}, {lon})" for lat, lon in zip(usgs_index_lat, usgs_index_lon)]


usgs_points_gdf = gpd.GeoDataFrame(
    {
        "id": usgs_index_filter['COMID'],
        "coord": usgs_index_coord,
        "lat": usgs_index_lat,
        "lon": usgs_index_lon,
    },
    geometry=gpd.points_from_xy(usgs_index_lon, usgs_index_lat),
    crs='EPSG:4269'
)
usgs_points_gdf

,id,coord,lat,lon,geometry
0,3585554,"(30.1005540472, -99.2830259334)",30.100554,-99.283026,POINT (-99.28303 30.10055)
1,1438379,"(29.933918594, -95.2338897638)",29.933919,-95.233890,POINT (-95.23389 29.93392)
2,3168874,"(28.0593639757, -98.0967707015)",28.059364,-98.096771,POINT (-98.09677 28.05936)
3,1440525,"(29.7488063747, -95.29054195760001)",29.748806,-95.290542,POINT (-95.29054 29.74881)
4,1520249,"(30.2595874939, -95.3025198277)",30.259587,-95.302520,POINT (-95.30252 30.25959)
...,...,...,...,...,...
471,1279694,"(32.9659536513, -96.9445361442)",32.965954,-96.944536,POINT (-96.94454 32.96595)
472,1293010,"(32.9600600806, -96.6144209866)",32.960060,-96.614421,POINT (-96.61442 32.96006)
473,1306815,"(32.9456882556, -97.583105668)",32.945688,-97.583106,POINT (-97.58311 32.94569)
475,5254557,"(32.8978980877, -96.2533123486)",32.897898,-96.253312,POINT (-96.25331 32.8979)


In [6]:
coords = flow_lat_lon['lat_lon']
idx = flow_lat_lon['idx']
site_id = flow_lat_lon['COMID']
#boundaries = tx_shp
#basin_field = 'REGION'
boundaries = huc6
basin_field = 'HUC_6'
coord_crs = boundaries.crs
existing_sensors = usgs_points_gdf
print(coord_crs)

EPSG:4269


In [7]:
# for geographic coord sys (lat,lon), longitude wrapping
# - create separate functions for geo and proj crs 
# for projected coord sys (x,y), filter out points out-of-bounds 

lat_list, lon_list = [], []

for s in coords:
    stripped = s.strip("()")
    lat, lon = map(float, stripped.split(","))

    # Fix 0–360 longitudes to -180 to 180
    if lon > 180:
        print(f"flag: {s} was out of bounds")
        lon = ((lon + 180) % 360) - 180

    lat_list.append(lat)
    lon_list.append(lon)

points_gdf = gpd.GeoDataFrame(
    {
        "id": site_id,
        "idx": idx,
        "coord": coords,
        "lat": lat_list,
        "lon": lon_list,
    },
    geometry=gpd.points_from_xy(lon_list, lat_list),
    crs=coord_crs
)

In [8]:
points_gdf

,id,idx,coord,lat,lon,geometry
0,5256723,26936,"(32.61868276306611, -95.50137088386384)",32.618683,-95.501371,POINT (-95.50137 32.61868)
1,5256353,26782,"(32.63013428977702, -95.51512370866801)",32.630134,-95.515124,POINT (-95.51512 32.63013)
2,5256289,26750,"(32.64981553314542, -95.53823507416166)",32.649816,-95.538235,POINT (-95.53824 32.64982)
3,5256227,26720,"(32.661436989062686, -95.5524962744192)",32.661437,-95.552496,POINT (-95.5525 32.66144)
4,5256215,26714,"(32.668785323658845, -95.54849766843817)",32.668785,-95.548498,POINT (-95.5485 32.66879)
...,...,...,...,...,...,...
68834,210053,371,"(26.02768204709921, -97.604964548574)",26.027682,-97.604965,POINT (-97.60496 26.02768)
68835,210055,372,"(26.02905084838759, -97.60635442599998)",26.029051,-97.606354,POINT (-97.60635 26.02905)
68836,210057,373,"(26.02920018003503, -97.60606247351534)",26.029200,-97.606062,POINT (-97.60606 26.0292)
68837,942110021,64897,"(26.166418341363304, -97.46932823893809)",26.166418,-97.469328,POINT (-97.46933 26.16642)


In [ ]:
#points_gdf.to_file('flowlines_centroid.shp',driver='ESRI Shapefile')

In [9]:
proj_crs = "EPSG:5070"


"""
Assign basin values to points using spatial join + nearest-basin fallback.
Includes CRS validation and safe reprojection.
"""

# -----------------------------
# 2. Spatial join 
# -----------------------------
points_with_basin = gpd.sjoin(
    points_gdf,
    boundaries[[basin_field, "geometry"]],
    how="left",
    predicate="within",
)

# -----------------------------
# 3. Nearest-basin assignment
# -----------------------------
missing_mask = points_with_basin[basin_field].isna()
n_missing = missing_mask.sum()

if n_missing > 0:
    print(f"Found {n_missing} gauges not inside any basin. Assigning nearest basin...")

    pts_proj = points_with_basin.loc[missing_mask].to_crs(proj_crs)
    basins_proj = boundaries.to_crs(proj_crs)

    for idx in pts_proj.index:
        geom = pts_proj.loc[idx].geometry
        dists = basins_proj.geometry.distance(geom)

        nearest_idx = dists.idxmin()

        # Assign basin attributes
        points_with_basin.at[idx, basin_field] = boundaries.at[nearest_idx, basin_field]

# Cleanup
if "index_right" in points_with_basin:
    points_with_basin = points_with_basin.drop(columns=["index_right"])

points_with_basin['id'] = points_with_basin['id'].astype(int)
points_with_basin

,id,idx,coord,lat,lon,geometry,HUC_6
0,5256723,26936,"(32.61868276306611, -95.50137088386384)",32.618683,-95.501371,POINT (-95.50137 32.61868),120100
1,5256353,26782,"(32.63013428977702, -95.51512370866801)",32.630134,-95.515124,POINT (-95.51512 32.63013),120100
2,5256289,26750,"(32.64981553314542, -95.53823507416166)",32.649816,-95.538235,POINT (-95.53824 32.64982),120100
3,5256227,26720,"(32.661436989062686, -95.5524962744192)",32.661437,-95.552496,POINT (-95.5525 32.66144),120100
4,5256215,26714,"(32.668785323658845, -95.54849766843817)",32.668785,-95.548498,POINT (-95.5485 32.66879),120100
...,...,...,...,...,...,...,...
68834,210053,371,"(26.02768204709921, -97.604964548574)",26.027682,-97.604965,POINT (-97.60496 26.02768),121102
68835,210055,372,"(26.02905084838759, -97.60635442599998)",26.029051,-97.606354,POINT (-97.60635 26.02905),121102
68836,210057,373,"(26.02920018003503, -97.60606247351534)",26.029200,-97.606062,POINT (-97.60606 26.0292),121102
68837,942110021,64897,"(26.166418341363304, -97.46932823893809)",26.166418,-97.469328,POINT (-97.46933 26.16642),121102


In [10]:
existing_sensors = existing_sensors.merge(
    points_with_basin[["id", "idx"]],
    on="id",
    how="left"
)
existing_sensors


,id,coord,lat,lon,geometry,idx
0,3585554,"(30.1005540472, -99.2830259334)",30.100554,-99.283026,POINT (-99.28303 30.10055),22523
1,1438379,"(29.933918594, -95.2338897638)",29.933919,-95.233890,POINT (-95.23389 29.93392),8638
2,3168874,"(28.0593639757, -98.0967707015)",28.059364,-98.096771,POINT (-98.09677 28.05936),22304
3,1440525,"(29.7488063747, -95.29054195760001)",29.748806,-95.290542,POINT (-95.29054 29.74881),9049
4,1520249,"(30.2595874939, -95.3025198277)",30.259587,-95.302520,POINT (-95.30252 30.25959),15315
...,...,...,...,...,...,...
450,1279694,"(32.9659536513, -96.9445361442)",32.965954,-96.944536,POINT (-96.94454 32.96595),6249
451,1293010,"(32.9600600806, -96.6144209866)",32.960060,-96.614421,POINT (-96.61442 32.96006),7147
452,1306815,"(32.9456882556, -97.583105668)",32.945688,-97.583106,POINT (-97.58311 32.94569),8556
453,5254557,"(32.8978980877, -96.2533123486)",32.897898,-96.253312,POINT (-96.25331 32.8979),26440


In [11]:
#existing_sensors = usgs_points_gdf
existing_sensors = points_with_basin[points_with_basin['id'].isin(existing_sensors['id'])]
existing_sensors

,id,idx,coord,lat,lon,geometry,HUC_6
46,5256789,26943,"(32.80666011177604, -95.91465184174388)",32.806660,-95.914652,POINT (-95.91465 32.80666),120100
60,5253949,26358,"(33.12949416198601, -96.06924293976255)",33.129494,-96.069243,POINT (-96.06924 33.12949),120100
95,5254557,26440,"(32.89376483968054, -96.24023998781136)",32.893765,-96.240240,POINT (-96.24024 32.89376),120100
491,5255921,26567,"(32.77245916630714, -95.79087681427212)",32.772459,-95.790877,POINT (-95.79088 32.77246),120100
837,5278868,27896,"(32.331875490217215, -94.35467070522385)",32.331875,-94.354671,POINT (-94.35467 32.33188),120100
...,...,...,...,...,...,...,...
67459,1636035,19040,"(27.71379981829267, -97.51000076413202)",27.713800,-97.510001,POINT (-97.51 27.7138),121102
67546,1653465,19928,"(27.772966029947284, -98.03473009589644)",27.772966,-98.034730,POINT (-98.03473 27.77297),121102
67578,1653097,19801,"(27.762555252828797, -98.10398847592883)",27.762555,-98.103988,POINT (-98.10399 27.76256),121102
67604,1655121,19958,"(27.521178595776767, -97.8430981372013)",27.521179,-97.843098,POINT (-97.8431 27.52118),121102


In [12]:
"""
Quotas = how many existing sensors fall in each basin.

existing_pts = gpd.sjoin(existing_sensors,
          boundaries[[basin_field,"geometry"]],
          how='left',
          predicate='within')
"""
# quotas_from_existing
existing_pts = gpd.sjoin(existing_sensors,
          boundaries[["geometry"]],
          how='left',
          predicate='within')
if "index_right" in existing_pts:
    existing_pts = existing_pts.drop(columns=["index_right"])
quotas = existing_pts.groupby(basin_field).size().to_dict()
quotas

{'120100': 35,
 '120200': 18,
 '120301': 53,
 '120302': 16,
 '120401': 53,
 '120402': 7,
 '120500': 9,
 '120601': 16,
 '120602': 25,
 '120701': 22,
 '120702': 20,
 '120800': 9,
 '120901': 29,
 '120902': 25,
 '120903': 8,
 '120904': 1,
 '121001': 6,
 '121002': 28,
 '121003': 27,
 '121004': 9,
 '121101': 26,
 '121102': 5}

In [13]:
selected_rows = []
selected_sensor_idx = []

# grp: candidate grid points in basin
basin, grp = next(iter(points_with_basin.groupby(basin_field)))
print(f"basin and grp = {basin}, {len(grp)}")
k = quotas.get(basin,0)
print(f"k={k}")

basin_idx = grp['idx'].to_numpy()
basin_id = grp['id'].to_numpy()
print(f"basin_idx {len(basin_idx)}")

k = min(k, len(basin_idx))


id_to_idx = dict(zip(basin_id, basin_idx))

# Find fixed sensors inside basin
fixed_idx = []
fixed_id = []
if existing_sensors is not None:
    mask = existing_sensors['id'].isin(basin_id)
    fixed_id = existing_sensors.loc[mask,'id'].tolist()
    fixed_idx = existing_sensors.loc[mask,'idx'].tolist()

print(f"fixed_indices {len(fixed_idx)}")

free_id  = [cid for cid in basin_id if cid not in fixed_id]
free_idx = [id_to_idx[cid] for cid in free_id]
print(f"free_indices {len(free_idx)}")

basin and grp = 120100, 5290
k=35
basin_idx 5290
fixed_indices 35
free_indices 5255


In [14]:
fixed_idx = []
# QR with fixed sensors
if fixed_idx:
    print('wenus')
    A_F = X_train[:, fixed_idx]
    A_R = X_train[:, free_idx]
    QF, _ = np.linalg.qr(A_F)
    projection = QF @ (QF.T @ A_R)
    A_R_prime = A_R - projection

    _, _, pivots = qr(A_R_prime, mode="economic", pivoting=True)
    free_n = max(0, k - len(fixed_idx))

    chosen_idx = fixed_idx + free_idx[pivots[:free_n]]
else:
    print('beengus')
    _, _, pivots = qr(X_train[:,basin_idx], mode="economic", pivoting=True)
    #_, _, piv = qr(X_train,mode='economic',pivoting=True) # piv same as chosen_idx
    chosen_idx = basin_idx[pivots[:k]]

selected_sensor_idx.extend(chosen_idx)
sel = (grp.set_index('idx').loc[chosen_idx].reset_index()[[basin_field,"id","idx","lat","lon"]])
selected_rows.append(sel)

beengus


In [20]:
sel[:5]

,REGION,id,idx,lat,lon
0,Texas-Gulf Region,24719331,64623,29.704796,-93.848520
1,Texas-Gulf Region,1440477,9025,29.763342,-95.079982
2,Texas-Gulf Region,1486152,12808,31.088417,-95.743734
3,Texas-Gulf Region,3124672,22021,29.266256,-95.572427
4,Texas-Gulf Region,1114367,2009,30.095373,-94.088387


In [21]:
selected_rows[0][:5]

,REGION,id,idx,lat,lon
0,Texas-Gulf Region,24719331,64623,29.704796,-93.848520
1,Texas-Gulf Region,1440477,9025,29.763342,-95.079982
2,Texas-Gulf Region,1486152,12808,31.088417,-95.743734
3,Texas-Gulf Region,3124672,22021,29.266256,-95.572427
4,Texas-Gulf Region,1114367,2009,30.095373,-94.088387


In [22]:
pivots[:5]

array([21989, 21005, 17253, 34207,  8107], dtype=int32)

In [23]:
basin_idx[pivots[0]]

np.int64(64623)

In [24]:
chosen_idx[:5]

array([64623,  9025, 12808, 22021,  2009])

In [25]:
grp.loc[grp["idx"].isin([basin_idx[pivots[0]]])]

,id,idx,coord,lat,lon,geometry,REGION
23409,24719331,64623,"(29.704796493737533, -93.84852001337936)",29.704796,-93.84852,POINT (-93.84852 29.7048),Texas-Gulf Region


In [26]:
df_cleaned.columns.values[basin_idx[pivots[0]]]

'24719331'

In [27]:
from sensor_network_utils import *
# Prepare USGS indices
usgs_location, usgs_number, columns_to_keep = prepare_usgs_indices(df_cleaned, usgs_index_df)

# Split data
X_train, X_test = split_data(df_cleaned.values)

# Get sensor placement from QR
sensor_location = sensor_placement_qr(X_train, usgs_number)

In [28]:
sensor_location[:5]

array([64623,  9025, 12808, 22021,  2009], dtype=int32)

In [30]:
sum(x != y for x, y in zip(selected_rows[0]['idx'], sensor_location))

np.int64(0)

In [ ]:
# update code
# recon
# performance metric
# visualization

In [69]:
# recon
'''
key points
- QR chooses where to place sensors
- reconstruction_evaluation tests how good that placement is by rearranging the data matrix using only the time series from observed sensors
- X(t,:) ~ XJ(t,:)T, each column of T tells how to combine sensors to reconstruct one location
outputs
- X_test_selected: actual observed data at sensors
- X_test_recon: full reconstructed matrix
- selected_sensors: indices of sensors used
- non_selected_sensors: indices of ungauged sensors
- rmse: per-reach error
- relative_error: basin-wide performance
'''
'''
return dataclass 
    result.X_hat
    result.rmse
    result.relative_error
    result.recon
or return_ = 'metrics', 'recon', 'all'
'''
# input
n_sensors = len(selected_rows[0])
sensor_location = selected_rows[0]['idx']

N_sensors = X_test.shape[1]
all_sensors = np.arange(N_sensors)
selected_sensors = sensor_location[:n_sensors]
non_selected_sensors = np.setdiff1d(all_sensors, selected_sensors)

X_train_selected = X_train[:, selected_sensors]  
X_test_selected = X_test[:, selected_sensors]

solution = np.linalg.lstsq(X_train_selected.T, X_test_selected.T, rcond=None)[0] # eq 7, for each location finds weights that best express it using sensor signals
X_test_reconstructed = solution.T @ X_train
X_test_reconstructed = np.maximum(X_test_reconstructed, 1e-10)

rmse = np.sqrt(np.mean((X_test - X_test_reconstructed) ** 2, axis=0))
relative_error = np.linalg.norm(X_test_reconstructed - X_test,'fro') / np.linalg.norm(X_test,'fro')
print(f"rmse: {rmse.mean()}, rel_err: {relative_error}")

rmse: 8.786422641915792, rel_err: 0.7232455766539768


In [70]:
# metric performance
# inputs
true_values = X_test
pred_values = X_test_reconstructed

ss_res = np.sum((true_values - pred_values) ** 2, axis=0)
ss_tot = np.sum((true_values - np.mean(true_values, axis=0)) ** 2, axis=0)

with np.errstate(divide='ignore', invalid='ignore'):
    r_squared = np.where(ss_tot != 0, 1 - (ss_res / ss_tot), np.nan)
    nse = np.where(ss_tot != 0, 1 - (ss_res / ss_tot), np.nan)
    nnse = 1 / (2 - nse)